We are trying to change the X,Y datapoints into polynomails.

Step 1: Read Curve XML into DF

In [1]:
#### IMPORTS #####
import xml.etree.ElementTree as ET
import pandas as pd
import typing as typ
import numpy as np
###################

#### Curve-to-XML Reader ####

#Model = 2095 KP -> Curve Groups [Fixed Speed / Bronze as one example] -> Curve Trim = 4.29" plus all the additional curves 

class Model():
    def __init__(self,pumpsize,pumpmodel):
        self.pumpSize=pumpsize #adding hydraulic model name
        self.pumpModel=pumpmodel
        #self.xpath_dict={} # Lets make the x-path dict
        self.PLO_dict={} #Lets make the productline options dict that contains the PLO and a list of options for each PLO an example would be imp material that has SS and Bronze
        #self.CurveGroups_dict={} # Lets Create a Curve Group
        self.CurveGroups=[]
        self.name=self.__repr__()

    def __repr__(self):
        return f"{self.pumpModel} {self.pumpSize}"
    
    def add_xpath_to_dict(self,key:str,xpath:str)->None:
        """This adds an item to the xpath dictionary that can be used by all child methods."""
        self.xpath_dict.update({key:xpath})

    def add_to_CurveGroups_dict(self,key:str,cg):
        """This will add a CurveGroup object to the dict"""
        self.CurveGroups_dict.update({key:cg})
    
    def add_to_PLO_dict(self,key,value):
        """This will add to the PLO_dict."""
        if not bool(self.PLO_dict):
            self.PLO_dict.update({key:{value}})
        else:
            self.PLO_dict[key].add(value)
       
class CurveGroup():
    def __init__(self,curve_number):
        self.CurveList=[] #Lets add a list of curves
        self.PLO_dict={} #Adds a PLO dict for the options that are used for this Curve Groups 
        self.CurveList=[] #Lets add a list of curve objects into this list  
        self.curve_number=curve_number #Lets have the parent curve number
    
    def __repr__(self):
        return self.curve_number

class Curve():
    """
    Example:
    CurveType
        X           Y
        [1,2,3]     [1,2,3]
    Poles: Number of Poles the Pump is running at i.e 2,4,6, etc. 
    Trim: Trim of the curve
    HP: End of Curve HP
    """
    def __init__(self,poles,trim,parent,curve_xpath):
        self.poles=poles #What is the HP MAX
        self.curve_dict={'Head':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'NPSH':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'Efficiency':{'Coeff':[],'XY-Points':{'x':[],'y':[]}},'Power':{'Coeff':[],'XY-Points':{'x':[],'y':[]}}}
        self.xpath_dict=curve_xpath
        self.trim=trim #What is the trim on these curves
        self.parent=parent
        self.name=self.__repr__()

    def __repr__(self):
        return f"{self.parent}/{self.trim}"
    
    def __round_to_HPStep(self,HP,tol=.5):
        """Takes a calculated HP and rounds it to the correct motor step."""
        STEPS = [0.5, 0.75,1, 1.5, 2, 3, 5, 7.5, 10, 15, 20, 25, 30, 40, 50, 60, 75, 100, 125, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1250, 1500, 1750, 2000]
        for step in STEPS:
            if step >= HP or abs(HP-step)<=tol:
                return step
        
    def add_curve_to_dict(self,X_list,Y_list):
        pass
    def add_xpath_to_dict(self):
        pass
    
    def get_xy_points(self,CurveType,xml_ele):
        #Lets get the x-y points
        points=[]
        for curve in xml_ele.find(f"{self.xpath_dict[CurveType]}"):
            points.append((curve.find("./x").text,curve.find("./y").text))
        x,y=zip(*points)
        x=[float(i) for i in x]
        y=[float(i) for i in y]
        return x,y
    
    def getcurvepolynomial(self,deg,CurveType,xml_ele):
        """This returns the polynomial coefficients from the """
        x,y=self.get_xy_points(CurveType,xml_ele)
        coef=np.polyfit(x,y,deg)
        return coef
    
    def complete_curve_dict(self,xml_ele):
        for k,v in self.curve_dict.items():
            try:
                self.curve_dict[k]['Coeff']=self.getcurvepolynomial(5,k,xml_ele)
                self.curve_dict[k]['XY-Points']['x'],self.curve_dict[k]['XY-Points']['y']=self.get_xy_points(k,xml_ele)
                if k=='Head':
                    x_max=self.curve_dict[k]['XY-Points']['x'][-1]
                    y_max=self.curve_dict[k]['XY-Points']['y'][-1]
                    p_kw=(9810/(3.6*(10**6)))*x_max*y_max
                    p_hp=round(p_kw/0.746,2)
                    self.HP=self.__round_to_HPStep(p_hp)
            except:
                f"{k} does not exist in the XML" #add to a logger
    
        




















In [22]:
### Testing ###

filename=r'C:\Users\100477\OneDrive - Grundfos\NK\KP_PL.xml'

# Tree
tree=ET.parse(filename)


#Need a function to create the x-path dict or I need a dict that extends from the dict class but reserves the key words.

#root, model, CompatRule are reserved words: 
xpath_dict={"root":"//pumpCommon",
            'supplierID':'.//supplierID',
            'model':'.//pumpSizeAndType',
            'SolidsLimit':'.//solidsDiaLimit',
            'SucSize':'.//diaNozzleSuction',
            'DisSize':'.//diaNozzleDischarge',
            'MinSpeed':'.//speedLimitMin',
            'MaxSpeed':'.//speedLimitMax',
            'powerLimit':'.//powerShaftLimit',
            'TempLimit':'.//maximumTemperatureRise',
            'CompatRule':'.//compatibilityRule',
            'curvenumber':'.//curveNumber'
            }
map_dict={'SolidsLimit':''} # For column names
# We need to create the models

def instance_exists(d:dict, attr_name, attr_value):
    for instance in d.values():
        if getattr(instance, attr_name) == attr_value:
            return True
    return False


def make_models(tree:ET.ElementTree,xpath_dict:dict):
    models=tree.findall(xpath_dict['root'])
    model_dict={}
    for model in models:
        #Check that the status is active
        if model.attrib['status']=='active':
            pumpSize,pumpModel=model.find(xpath_dict['model']).text.split(" ")
            hyd_model=Model(pumpmodel=pumpModel,pumpsize=pumpSize) #Making the Model
            for k,v in xpath_dict.items():
                if k=='root' or k=='model':
                    continue
                elif k=='curvenumber':
                    hyd_model.CurveGroups=[CurveGroup(curve_number.text) for curve_number in model.findall(v)]
                else:
                    k,model.find(v).text #Make a DataFrame
            model_dict.update({hyd_model.name:hyd_model})        
    return model_dict
            

a=make_models(tree,xpath_dict)
b=a['KP 1012-1/2'].CurveGroups[0]
b.curve_number

C:\Users\100477\AppData\Local\Temp\ipykernel_28516\1826818349.py:36: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/pumpCommon'
  models=tree.findall(xpath_dict['root'])


'RC2332-5'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Lets start at Curve Level
tree_curve=ET.parse('KP.xml')

xpath_dict={
    'trim':'./Impeller/diameter',
    'poles':'./polesRef',
    'name':'./curveNumber'    
} #This is input for CurveGroup

curve_xpath_dict={
    'Head':"./Impeller/Curve[@type='Head']",
    'Efficiency':"./Impeller/Curve[@type='Efficiency']",
    'NPSH':"./Impeller/Curve[@type='NPSH']",
    'Power':"./Impeller/Curve[@type='Power']"
}

curves_dict={} #This is a list of curves
for curveID in tree_curve.findall('.//pumpCurveCollection'): # This finds all pump curve collection
    #This creates the curve object
    pole=int(curveID.find(xpath_dict['poles']).text)
    parent=curveID.find(xpath_dict['name']).text
    for trim in curveID.findall(xpath_dict['trim']):
        curve_obj=Curve(pole,float(trim.text),parent,curve_xpath_dict)
        curve_obj.complete_curve_dict(curveID)
        print(parent,trim.text)
    curves_dict.update({curve_obj.name:curve_obj})
    







In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Lets start at Curve Level


# Create a polynomial object with the coefficients
polynomial = np.poly1d(curves_list[1].curve_dict['Head']['Coeff'])

# Generate x values for plotting the polynomial curve
x_fit = np.linspace(curves_list[1].curve_dict['Head']['XY-Points']['x'][0], curves_list[1].curve_dict['Head']['XY-Points']['x'][-1], 100)
y_fit = polynomial(x_fit)

x=curves_list[1].curve_dict['Head']['XY-Points']['x']
y=curves_list[1].curve_dict['Head']['XY-Points']['y']

# Plot the data points and the fitted curve
plt.scatter(x, y, color='red', label='Data Points')
plt.plot(x_fit, y_fit, color='blue', label='5th Order Polynomial Fit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('5th Order Polynomial Fit to Data Points')
plt.show()





In [ ]:
curves_dict['ID13759-3/191.77'].HP